In [ ]:
!pip install -q transformers accelerate datasets trl peft bitsandbytes sentencepiece evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.6/504.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount("/content/drive")   # run → follow the auth link → paste the code


Mounted at /content/drive


In [ ]:

import os
from pathlib import Path
import torch
import torch.nn.functional as F
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForCausalLM,
                          TrainingArguments, Trainer)
from peft import get_peft_model, LoraConfig, TaskType
from trl import PPOTrainer, PPOConfig


teacher_model_id   = "Salesforce/codegen-6B-mono"
student_model_id   = "Salesforce/codegen-350M-multi"
save_root          = Path("/content/drive/MyDrive/bachelor_project")          # change if you want another folder

sft_epochs         = 3
sft_lr             = 5e-5

distil_steps       = 2000               # enlarge for real run
distil_lr          = 5e-5
distil_T           = 2.0                # temperature for KL

ppo_steps          = 200                # # optimisation steps
ppo_lr             = 1e-5

device             = "cuda" if torch.cuda.is_available() else "cpu"
bf16               = torch.cuda.is_available()  # True on modern GPUs


In [ ]:
# ---- teacher (same tokenizer family) ----
# teacher_name =  "Salesforce/codegen-6B-mono"     # or 16B if you have VRAM

teacher = (AutoModelForCausalLM
               .from_pretrained(teacher_model_id,
                                torch_dtype=torch.bfloat16 if bf16 else None,
                                device_map="auto").eval())

print("Teacher ready (CodeGen family, 4-bit)")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/997 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/14.3G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/14.3G [00:00<?, ?B/s]

Some weights of the model checkpoint at Salesforce/codegen-6B-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.20.attn.causal_mask', 'transformer.h.21.attn.causal_mask', 'transformer.h.22.attn.causal_mask', 'transformer.h.23.attn.causal_mask', 'transformer.h.24.attn.causal_mask', 'transformer.h.25.attn.causal_mask', 'transformer.h.26.attn.causal_mask', 'transformer.h.27.attn.causal_mask', 'transformer.h.28.attn.causal_mask', 'transformer.h.29.attn.causal_mask', 'transformer.h.3.attn.causal_mas

Teacher ready (CodeGen family, 4-bit)


In [ ]:
# student_name = "Salesforce/codegen-350M-multi"   # NL + code
tok_s = AutoTokenizer.from_pretrained(student_model_id)
if tok_s.pad_token is None:
    tok_s.pad_token = tok_s.eos_token
tok_s.padding_side = "right"

student_base = AutoModelForCausalLM.from_pretrained(
    student_model_id,
    torch_dtype=torch.bfloat16 if bf16 else None,
    device_map="auto",
)

# peft_cfg = LoraConfig(
#         task_type=TaskType.CAUSAL_LM,
#         r=8,
#         lora_alpha=32,
#         lora_dropout=0.05,
#         bias="none",
#         target_modules=["qkv_proj","out_proj","fc_in","fc_out",  ],
#     )
# student = get_peft_model(student_model_id).to(device)
# student.print_trainable_parameters()

# student = get_peft_model(student_base, lora_cfg)
# student.print_trainable_parameters()

Some weights of the model checkpoint at Salesforce/codegen-350M-multi were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (

trainable params: 2,621,440 || all params: 359,333,888 || trainable%: 0.7295


In [ ]:
# ---------------------- load NL→code data -----------------------
# ---------------------- load NL→code data -----------------------
from datasets import load_dataset

def load_code_alpaca():
    try:
        return load_dataset("HuggingFaceH4/code_alpaca_20k", split="train")
    except Exception:
        url = "https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json"
        return load_dataset("json", data_files=url, split="train")

ds = load_code_alpaca().shuffle(seed=400)      # ← built-in shuffle
ds = ds.select(range(1000))             # keep only 400 examples

pairs = [
    (
        ex["instruction"] + ("\n" + ex["input"] if ex.get("input") else ""),
        ex["output"]
    )
    for ex in ds
]


def make_prompt(inst, code):
    return f"### Instruction:\\n{inst}\\n\\n### Response:\\n{code}"

def chunks(lst, n):            # batching helper
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

"qkv_proj",           # attention Q-K-V projection
"out_proj",           # attention outpu
"fc_in",              # MLP up-projection
"fc_out",

('fc_out',)

In [ ]:
# ---------------- extra supervised fine-tune -------------------
from torch.utils.data import DataLoader

# Re-use the `pairs` list (prompt, code) from earlier
class AlpacaCodeDataset(torch.utils.data.Dataset):
    def __len__(self): return len(pairs)
    def __getitem__(self, i):
        prompt, answer = pairs[i]
        text = f"{prompt}\n{answer}"
        toks = tok_s(text,
                     padding="max_length",
                     max_length=128,
                     truncation=True,
                     return_tensors="pt")
        toks["labels"] = toks["input_ids"].clone()
        return {k: v.squeeze(0) for k, v in toks.items()}

loader = DataLoader(AlpacaCodeDataset(), batch_size=4, shuffle=True)

opt = torch.optim.AdamW(student_base.parameters(), lr=5e-5)

student_base.train()
for step, batch in enumerate(loader, 1):
    batch = {k: v.to(student_base.device) for k, v in batch.items()}
    loss  = student_base(**batch).loss
    loss.backward()
    torch.nn.utils.clip_grad_norm_(student_base.parameters(), 1.0)
    opt.step(); opt.zero_grad()
    if step % 25 == 0:
        print(f"Sup-tune step {step:>3} | LM loss {loss.item():.3f}")
    if step == 200:          # 100 gradient steps ≈ a few minutes
        break
print("✅ Extra supervised pass complete")


Sup-tune step  25 | LM loss 2.906
Sup-tune step  50 | LM loss 1.578
Sup-tune step  75 | LM loss 1.531
Sup-tune step 100 | LM loss 2.406
Sup-tune step 125 | LM loss 0.941
Sup-tune step 150 | LM loss 2.844
Sup-tune step 175 | LM loss 1.672
Sup-tune step 200 | LM loss 0.840
✅ Extra supervised pass complete


In [ ]:
# --------------------- distillation loop ------------------------
import gc

# ---- teacher (same tokenizer family) ----
teacher = (AutoModelForCausalLM
               .from_pretrained(teacher_model_id,
                                torch_dtype=torch.bfloat16 if bf16 else None,
                                device_map="auto").eval())

opt = torch.optim.AdamW(student_base.parameters(), lr=1e-5)
kl = torch.nn.KLDivLoss(reduction="batchmean")

step = 0
for batch in chunks(pairs, 16):
    prompts = [make_prompt(i, c) for i, c in batch]

    t_inp = tok_s(prompts, return_tensors="pt",
                  padding=True, truncation=True,
                  max_length=128).to(teacher.device)
    with torch.no_grad():
        t_probs = torch.softmax(teacher(**t_inp).logits.float(), dim=-1)

    s_inp = tok_s(prompts, return_tensors="pt",
                  padding=True, truncation=True,
                  max_length=128).to(student_base.device)
    s_logp = torch.log_softmax(student_base(**s_inp).logits, dim=-1)

    loss = kl(s_logp, t_probs)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(student_base.parameters(), 1.0)
    opt.step(); opt.zero_grad()

    step += 1
    if step % 25 == 0:
        print(f"Distill {step}/{200} | KL {loss.item():.4f}")
    if step >= 200:
        break

print("✅ Distillation done")
del teacher; gc.collect(); torch.cuda.empty_cache()

Some weights of the model checkpoint at Salesforce/codegen-6B-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.20.attn.causal_mask', 'transformer.h.21.attn.causal_mask', 'transformer.h.22.attn.causal_mask', 'transformer.h.23.attn.causal_mask', 'transformer.h.24.attn.causal_mask', 'transformer.h.25.attn.causal_mask', 'transformer.h.26.attn.causal_mask', 'transformer.h.27.attn.causal_mask', 'transformer.h.28.attn.causal_mask', 'transformer.h.29.attn.causal_mask', 'transformer.h.3.attn.causal_mas

Distill 25/200 | KL 63.3123
Distill 50/200 | KL 62.9108
✅ Distillation done


In [ ]:
# Drive is already mounted, so just go on and save:
from pathlib import Path

SAVE_DIR = Path("/content/drive/MyDrive/bachelor_project/pre_sup")
SAVE_DIR.mkdir(parents=True, exist_ok=True)   # create folder if missing

student_base.save_pretrained(SAVE_DIR)
tok_s.save_pretrained(SAVE_DIR)

print("✅ Student weights & tokenizer written to", SAVE_DIR)


✅ Student weights & tokenizer written to /content/drive/MyDrive/bachelor_project/pre_sup


In [ ]:
# assume `student` is your PEFT model in memory
merged = student.merge_and_unload()      # merges LoRA into the base
MERGED_DIR = "/content/drive/MyDrive/bachelor_project/student_codegen350M_merged"
merged.save_pretrained(MERGED_DIR, safe_serialization=True)
tok_s.save_pretrained(MERGED_DIR)

('/content/drive/MyDrive/bachelor_project/student_codegen350M_merged/tokenizer_config.json',
 '/content/drive/MyDrive/bachelor_project/student_codegen350M_merged/special_tokens_map.json',
 '/content/drive/MyDrive/bachelor_project/student_codegen350M_merged/vocab.json',
 '/content/drive/MyDrive/bachelor_project/student_codegen350M_merged/merges.txt',
 '/content/drive/MyDrive/bachelor_project/student_codegen350M_merged/added_tokens.json',
 '/content/drive/MyDrive/bachelor_project/student_codegen350M_merged/tokenizer.json')

In [ ]:
!pip install -qU faiss-cpu sentence-transformers datasets          # 1× per Colab

from datasets import load_dataset, concatenate_datasets
from sentence_transformers import SentenceTransformer
import faiss, numpy as np, json, gzip, os, re, textwrap

from datasets import load_dataset

# pick ONE of these -------------------------------------

corpus_ds = load_dataset("mbpp", split="train")

# build corpus list expected by our RAG builder ----------
def make_chunks(row):
    code = row.get("code") or row.get("content") or row.get("snippet") or ""
    doc  = row.get("docstring", "")
    fn   = row.get("func_name") or row.get("path") or row.get("task_id")
    text = f"{fn}\n{doc}\n```python\n{code}\n```"
    return {"text": text, "id": fn}

corpus = [make_chunks(r) for r in corpus_ds]
print("Corpus size:", len(corpus))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 35.0 MB/s eta 0:00:00


README.md: 0.00B [00:00, ?B/s]

full/train-00000-of-00001.parquet:   0%|          | 0.00/87.2k [00:00<?, ?B/s]

full/test-00000-of-00001.parquet:   0%|          | 0.00/116k [00:00<?, ?B/s]

full/validation-00000-of-00001.parquet:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/374 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/90 [00:00<?, ? examples/s]

Generating prompt split:   0%|          | 0/10 [00:00<?, ? examples/s]

Corpus size: 374


In [ ]:
import pathlib, os
OUT_DIR = pathlib.Path('/content/drive/MyDrive/bachelor_project')
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
embedder   = SentenceTransformer("jinaai/jina-embeddings-v2-base-code")
dim        = embedder.get_sentence_embedding_dimension()
index      = faiss.IndexFlatIP(dim)

batch = 128
all_vecs, ids = [], []
for i in range(0, len(corpus), batch):
    texts = [c["text"][:512] for c in corpus[i:i+batch]]   # trim long chunks
    vecs  = embedder.encode(texts, convert_to_numpy=True, normalize_embeddings=True)
    all_vecs.append(vecs)
    ids.extend(range(i, i+len(texts)))
index.add(np.vstack(all_vecs))
idx_file = OUT_DIR / "rag_index.faiss"
json_file = OUT_DIR / "rag_corpus.json"

faiss.write_index(index, str(idx_file))
json.dump(corpus, open(json_file, "w"))

print("✅  Saved index to", idx_file)
print("✅  Saved corpus JSON to", json_file)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/322M [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


✅  Saved index to /content/drive/MyDrive/bachelor_project/rag_index.faiss
✅  Saved corpus JSON to /content/drive/MyDrive/bachelor_project/rag_corpus.json


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import AutoPeftModelForCausalLM

# paths
STUDENT_DIR   = "/content/drive/MyDrive/bachelor_project/student_codegen350M_merged"
INDEX_FILE    = "/content/drive/MyDrive/bachelor_project/rag_index.faiss"
CORPUS_FILE   = "/content/drive/MyDrive/bachelor_project/rag_corpus.json"

# load student
tok = AutoTokenizer.from_pretrained(STUDENT_DIR, trust_remote_code=True)
lm  = AutoModelForCausalLM.from_pretrained(
        STUDENT_DIR, device_map="auto", torch_dtype="auto").eval()

# load retrieval objects
index  = faiss.read_index(INDEX_FILE)
corpus = json.load(open(CORPUS_FILE))

def retrieve(query, k=5):
    qv  = embedder.encode([query], normalize_embeddings=True)
    D,I = index.search(qv, k)
    return [corpus[i]["text"] for i in I[0]]

def rag_generate(instruction, k=5, max_tokens=180):
    ctx_chunks = retrieve(instruction, k)
    context    = "\n\n".join(ctx_chunks)
    prompt = f"""\
### Context:
{context}

### Instruction:
{instruction}

### Response:
"""
    ids = tok(prompt, return_tensors="pt", truncation=True,
              max_length=2048 - max_tokens).to(lm.device)
    gen = lm.generate(**ids, max_new_tokens=max_tokens,
                      temperature=0.2, top_p=0.95,
                      eos_token_id=tok.eos_token_id, pad_token_id=tok.eos_token_id)
    return tok.decode(gen[0], skip_special_tokens=True).split("### Response:")[-1].strip()


In [ ]:
pip install -qU sentence-transformers faiss-cpu rank_bm25


In [ ]:
# ╔═══════════════  0 · SET-UP  ═══════════════════════════════════╗
!pip install -qU "transformers>=4.38" peft accelerate faiss-cpu sentence-transformers

import ast, json, torch, textwrap, pprint
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft         import AutoPeftModelForCausalLM
from sentence_transformers import SentenceTransformer
import faiss, numpy as np


# ╔═══════════════  1 · MODEL PATHS  ══════════════════════════════╗
MODEL_PATHS = {
    # "base-350M"   : "Salesforce/codegen-350M-mono",
    # "teacher-2B"  :"Salesforce/codegen-6B-mono",    # or 16B if you have VRAM
    # "student-SFT" : "/content/drive/MyDrive/bachelor_project/pre_sup",
    "student-PPO" : "/content/drive/MyDrive/bachelor_project/student_codegen350M_merged",
}

# Which models should **use** retrieval?
USE_RAG = {"student-SFT"}          # set() → none, {"all"} → all

# ╔═══════════════  2 · RAG OBJECTS  ══════════════════════════════╗
# 🔧  Edit these two paths to wherever you saved them
IDX_FILE   = "/content/drive/MyDrive/bachelor_project/rag_index.faiss"
CORPUS_FILE= "/content/drive/MyDrive/bachelor_project/rag_corpus.json"

# load only once
print("⏳ loading FAISS index & corpus …")
index   = faiss.read_index(IDX_FILE)
corpus  = json.load(open(CORPUS_FILE))
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
import numpy as np, faiss, re, itertools

# -------- 1 · new encoder -------------------------------------------
embedder = SentenceTransformer("jinaai/jina-embeddings-v2-base-code")   # or -large if you have RAM/GPU

def embed(texts):
    return embedder.encode(texts, normalize_embeddings=True,
                           batch_size=64, convert_to_numpy=True)

# -------- 2 · hybrid retrieval --------------------------------------
TOK = re.compile(r"[A-Za-z_][A-Za-z0-9_]*")

# pre-tokenise corpus for BM25
bm25 = BM25Okapi([TOK.findall(chunk["text"]) for chunk in corpus])

def retrieve_hybrid(query, k_embed=20, k_final=6):
    # (a) semantic top-k
    q_vec   = embed([query])
    D,I     = index.search(q_vec, k_embed)
    cand_ix = I[0].tolist()
    cand_txt= [corpus[i]["text"] for i in cand_ix]

    # (b) BM25 rerank
    scores  = bm25.get_scores(TOK.findall(query))
    hybrid  = sorted(zip(cand_txt, scores[cand_ix]), key=lambda z: -z[1])

    return [txt for txt,_ in hybrid[:k_final]]

# -------- 3 · fenced prompt builder ---------------------------------
FENCE = "```python"
def build_prompt(ctx_chunks, instruction):
    ctx = "\n\n".join(ctx_chunks)
    return textwrap.dedent(f"""\
        ### Context:
        {FENCE}
        {ctx}
        {FENCE}

        ### Instruction:
        {instruction}

        ### Response:
        {FENCE}
        """)

def rag_generate(model, tok, instruction, k_final=6, max_tokens=160):
    chunks = retrieve_hybrid(instruction, k_final=k_final)
    prompt = build_prompt(chunks, instruction)
    ids    = tok(prompt, return_tensors="pt",
                 truncation=True, max_length=2048-max_tokens).to(model.device)

    out = model.generate(**ids,
                         max_new_tokens=max_tokens,
                         temperature=0.15, top_p=0.9,
                         eos_token_id=tok.convert_tokens_to_ids(FENCE),  # stop at ```python
                         pad_token_id=tok.eos_token_id,
                         repetition_penalty=1.1)
    answer = tok.decode(out[0], skip_special_tokens=True)
    # strip leading fence + trailing whitespace
    return answer.split(FENCE)[-1].strip()

# ╔═══════════════  3 · TEST PROMPTS  ═════════════════════════════╗
def contains(sub): return lambda t: float(sub.lower() in t.lower())
def valid_py(t):
    try: ast.parse(t); return 1.0
    except SyntaxError: return 0.0

TEST_ITEMS = [
    ("Write a Python function to compute the nth Fibonacci number.",
        lambda t: 0.5*contains("def")(t) + 0.5*valid_py(t)),
    # ("Generate a list comprehension that squares even numbers from 0-99.",
    #     lambda t: contains("[x**2 for x in range")(t)),
    ("write a python function for adding 2 numbers to eachother",
        lambda t: 0.33*contains("def")(t) + 0.33*valid_py(t) + 0.34*contains("+")(t)),
]

# ╔═══════════════  4 · MODEL LOADER  ═════════════════════════════╗
def load_model(path: str):
    if Path(path, "adapter_config.json").exists():
        mdl = AutoPeftModelForCausalLM.from_pretrained(
                  path, torch_dtype="auto", device_map="auto").eval()
    else:
        mdl = AutoModelForCausalLM.from_pretrained(
                  path, torch_dtype="auto", device_map="auto").eval()
    tok = AutoTokenizer.from_pretrained(path, trust__remote_code=True)
    return tok, mdl

# ╔═══════════════  5 · MAIN LOOP  ════════════════════════════════╗
stats = {n: {"scores": []} for n in MODEL_PATHS}

for name, path in MODEL_PATHS.items():
    tok, mdl = load_model(path)
    rag_flag = ("all" in USE_RAG) or (name in USE_RAG)

    for prompt, judge in TEST_ITEMS:
        if rag_flag:
            answer = rag_generate(mdl, tok, prompt)          # ← RAG branch
        else:
            ids = tok(f"### Instruction:\n{prompt}\n\n### Response:\n",
                      return_tensors="pt").to(mdl.device)
            out  = mdl.generate(**ids, max_new_tokens=160, temperature=0.0,
                                 do_sample=False, eos_token_id=tok.eos_token_id,
                                 pad_token_id=tok.eos_token_id)
            answer = tok.decode(out[0], skip_special_tokens=True
                     ).split("### Response:")[-1].strip()

        score = judge(answer)
        stats[name]["scores"].append(score)

        print(f"\n{'='*100}\nMODEL: {name}"
              f"\nRAG  : {'yes' if rag_flag else 'no'}"
              f"\nPROMPT: {prompt}\nSCORE : {score:.2f}"
              f"\nANSWER ↓\n{textwrap.indent(answer, '    ')}")

    scores = stats[name]["scores"]
    stats[name]["avg"] = sum(scores)/len(scores)

# ╔═══════════════  6 · SUMMARY  ══════════════════════════════════╗
print("\n\n===== FINAL SUMMARY =====")
pprint.pp(stats, compact=True, sort_dicts=False)

⏳ loading FAISS index & corpus …


Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-code and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.10.intermediate.dense.bias', 'encoder.layer.10.intermediate.dense.weight', 'encoder.layer.10.output.LayerNorm.bias', 'encoder.layer.10.output.LayerNorm.weight', 'encoder.layer.10.output.dense.bias', 'encoder.layer.10.output.dense.weight', 'encoder.layer.11.intermediate.dense.bias', 'encoder.layer.11.interme


MODEL: student-PPO
RAG  : no
PROMPT: Write a Python function to compute the nth Fibonacci number.
SCORE : 0.50
ANSWER ↓
    The function is given the nth Fibonacci number.

    ### Example:

    >>> def fib(n):
    ...     return n + 1

    ### Test cases:

    >>> fib(0)
    1

    ### References:

    - [Introduction to Python](http://docs.python.org/2/library/functions.html#functions.fib)
    - [Python 2.7](http://docs.python.org/2/library/functions.html#functions.fib)
    - [Python 2.6](http://docs.python.org/2/library/functions.html#functions.fib)
    - [Python 2.5](http://docs.python.

MODEL: student-PPO
RAG  : no
PROMPT: write a python function for adding 2 numbers to eachother
SCORE : 1.00
ANSWER ↓
    # Write a function that takes two numbers and adds them together.
    def add(x, y):
        return x + y

    # Write a function that takes two numbers and returns the sum.
    def sum(x, y):
        return x + y

    # Write a function that takes two numbers and returns the di